In [ ]:
import os
import sys
import socket
import re
import numpy as np
import string
import warnings
from timeit import default_timer as timer
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,desc,row_number,col,year,month,dayofmonth,dayofweek,to_timestamp,size,isnan,lit,lower
import pyspark.sql.functions as F
from pyspark.sql.types import MapType, StringType, IntegerType, StructType, StructField, FloatType, ArrayType

# Config

In [ ]:
country_code = "US"
country_name = "united-states"
language_code = "en"
print(country_code)

In [ ]:
try:
    spark
except NameError:
    if 'samuel' in socket.gethostname().lower():
        print('Create Local SparkSession')
        spark = SparkSession.builder.config(
        "spark.driver.host", "localhost").appName(
        "get-tweets-mentions").getOrCreate()
    else:
        print('Create Cluster SparkSession')
        spark = SparkSession.builder.appName(
        "get-tweets-mentions").getOrCreate()
    
# Local
print('Hostname:', socket.gethostname())
if  'samuel' in socket.gethostname().lower():
    path_to_data='../../data'
# Cluster
else:
    path_to_data='/user/spf248/twitter/data'
    
path_to_tweets=os.path.join(path_to_data,'decahose/parsed/tweets/tweets-with-identified-location-'+country_code)
path_to_timelines=os.path.join(path_to_data,'timelines','extract',country_name)
path_to_mentions=os.path.join(path_to_data,'mentions')
path_to_keywords=os.path.join(path_to_data,'keywords/labor/lang')
print(path_to_tweets)
print(path_to_timelines)
print(path_to_mentions)
print(path_to_keywords)

# Data Processing

In [ ]:
print('Import Datasets')
path_to_timelines='/user/spf248/twitter/data/timelines/extract/united-states/part-00199-ee33b2f6-08f4-49f4-9302-72a765ce509b-c000.snappy.parquet'
path_to_tweets='/user/spf248/twitter/data/decahose/parsed/tweets/tweets-with-identified-location-US/part-00199-1c5b6c3f-88eb-437c-80fc-c153b74934f5-c000.snappy.parquet'
tweets=spark.read.parquet(path_to_tweets)
timelines=spark.read.parquet(path_to_timelines)
df=tweets.unionByName(timelines)

In [ ]:
print("Cache")
df.cache()

In [ ]:
print("DROP DUPLICATES")
df=df.drop_duplicates(subset=['tweet_id'])

print("LOWERCASE")
df = df.withColumn('text',lower(col('text')))

print("SELECT LANGUAGE")
df = df.where(df.tweet_lang==language_code)

print("EXTRACT YEAR AND MONTH")
df = df.withColumn('year', year('created_at').cast("string"))
df = df.withColumn('month', month('created_at').cast("string"))
df = df.withColumn('day', dayofmonth('created_at').cast("string"))

In [ ]:
print('IMPORT MENTIONS')
mentions=spark.read.option('header','true').csv(os.path.join(path_to_keywords,language_code))
mentions=list(mentions.toPandas()['mention'])
print('# Mentions:',len(mentions))
print('\n'.join(mentions))

In [ ]:
print('LOOKUP MENTIONS')
for mention in mentions:
    field_mention='n_'+mention.replace(' ','_').replace('?','_')
    df=df.withColumn(field_mention, df.text.contains(mention).cast("int"))

In [ ]:
print('APPEND CONSTANT')
df=df.withColumn('n_tweets', lit(1))

df=df.drop('tweet_id','created_at','tweet_lang','place_id','tweet_longitude','tweet_latitude','text')

print("COUNT TWEETS AND MENTIONS BY YEAR, MONTH, LOCATION, AND USER")
df=df.groupBy('year','month','day','user_location','user_id').sum()

In [ ]:
print("TOTAL COUNTS")
print(df.groupBy().sum().collect())

In [ ]:
print('SAVE')
start = timer()

df.coalesce(1).write.mode("overwrite").json(os.path.join(path_to_mentions,country_code+'-json'))

end = timer()
print('DONE IN', round(end - start), 'SEC')